In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
import os
import re
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
R_shunt = 1
V_SS = 3.3

In [25]:
def calc_AmpereSecFromDataFrame(df):
    return integrate.trapezoid(df.current_A, df.time)

def calc_load_coulomb_str(df):
    load = calc_AmpereSecFromDataFrame(df)
    return "Q = {:.4f} As".format(load)

def get_model_execution_duration(df):
    return df.iloc[-1].time - df.iloc[0].time

In [26]:
def analyze_trace_file(path):
    df = pd.read_csv(path, names=["time", "signal_V", "trigger", "current_A"], sep=",", skiprows=1)
    df.current_A = df.signal_V / R_shunt
    #print(df.current_A.mean())
    df_active = df[df.trigger > 2]
    duration = get_model_execution_duration(df_active)
    charge_As = calc_AmpereSecFromDataFrame(df_active)
    charge_Ah = charge_As / 3600
    energy_Wh = charge_Ah * V_SS
    energy_Joule = energy_Wh * 3600
    return (duration, energy_Joule)

In [83]:
def analyze_subdirectory(measurement_dir_path):
    print("Measurement dir:", measurement_dir_path)
    df = pd.DataFrame(columns=["file", "energyJoule"])
    for filename in os.listdir(measurement_dir_path):
        if ".csv" not in filename:
            continue

        path = measurement_dir_path + filename
        (duration, energy_consumption) = analyze_trace_file(path)
        new_row = {'file':filename, 'energyJoule':energy_consumption, 'duration':duration}
        df = df.append(new_row, ignore_index=True)
        #print(path, "Energy consumption:", energy_consumption)

    return df


def analyze_directory(directory):
    dir_path = "/Users/thomas/Documents/RP2040/" + directory + "/"
    df_total = pd.DataFrame()
    for CR in [1, 2, 3, 4, 8, 16, 24, 32, 48, 64]:
        CR_percent = 0
        if CR > 0:
            CR_percent = 1 - (1 / CR)
            
        measurement_dir_path = dir_path + str(CR) + "/"
        if not os.path.exists(measurement_dir_path):
            continue
        
        df = analyze_subdirectory(measurement_dir_path)

        print("CR:", CR, "Energy:", df.energyJoule.mean(), "Duration:", df.duration.mean())
        new_row = {'CR':CR, 'CRPercent': CR_percent, 'energyJoule':df.energyJoule.mean(), 'duration': df.duration.mean(), 'energyJouleStd': df.energyJoule.std(), 'durationStd': df.duration.std()}
        df_total = df_total.append(new_row, ignore_index=True)
    
    return df_total

In [84]:
def fit_linear_function(x, y):
    reg = linear_model.LinearRegression()
    reg.fit(x, y)
    return reg.predict(x)

In [50]:
df_baseline = analyze_directory("baseline")

Measurement dir: /Users/thomas/Documents/RP2040/baseline/1/
CR: 1 Energy: 0.14859903981274167 Duration: 1.2915826000000001


In [96]:
df_unstructured = analyze_directory("unstructured_pruning")

Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/2/
CR: 2 Energy: 0.1462792868856847 Duration: 1.2894633333333334
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/3/
CR: 3 Energy: 0.14407205155364763 Duration: 1.289302733333333
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/4/
CR: 4 Energy: 0.1457294738219249 Duration: 1.2896025999999996
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/8/
CR: 8 Energy: 0.14686705946443712 Duration: 1.2967268000000005
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/16/
CR: 16 Energy: 0.1473396093893151 Duration: 1.3009938666666665
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/24/
CR: 24 Energy: 0.14748731901851805 Duration: 1.3031536000000001
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning/32/
CR: 32 Energy: 0.14906066964715145 Duration: 1.3039186666666664
Measurement dir: /Users/thomas/Documents/RP2040/unstr

In [92]:
df_structured = analyze_directory("structured_pruning")

Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/2/
CR: 2 Energy: 0.08065479237619931 Duration: 0.7130107333333332
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/3/
CR: 3 Energy: 0.07208352287112407 Duration: 0.6085825999999999
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/4/
CR: 4 Energy: 0.06092871970755538 Duration: 0.5175928666666667
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/8/
CR: 8 Energy: 0.04288140335035173 Duration: 0.3606255333333333
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/16/
CR: 16 Energy: 0.028350264870846624 Duration: 0.24684406666666664
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/24/
CR: 24 Energy: 0.01570750347778462 Duration: 0.13173639999999998
Measurement dir: /Users/thomas/Documents/RP2040/structured_pruning/32/
CR: 32 Energy: 0.015215989660989948 Duration: 0.1314553333333333
Measurement dir: /Users/thomas/Documents/RP2040/structured_

In [107]:
df_unstructured_speedup = analyze_directory("unstructured_pruning_speedup")

Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/1/
CR: 1 Energy: 0.1412214777398932 Duration: 1.3134998666666668
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/2/
CR: 2 Energy: 0.10664310173489629 Duration: 0.9353007999999998
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/3/
CR: 3 Energy: 0.09389962845217628 Duration: 0.8084036666666666
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/4/
CR: 4 Energy: 0.08765953036167988 Duration: 0.7450931333333333
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/8/
CR: 8 Energy: 0.07647189174585067 Duration: 0.6519040666666667
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/16/
CR: 16 Energy: 0.07307213997777924 Duration: 0.6042888000000001
Measurement dir: /Users/thomas/Documents/RP2040/unstructured_pruning_speedup/24/
CR: 24 Energy: 0.0703588494411698 Duration: 0.58865866666666

In [98]:
df_unstructured_with_baseline = df_baseline.append(df_unstructured, ignore_index=True)
df_structured_with_baseline = df_baseline.append(df_structured, ignore_index=True)

## Describe datasets

In [104]:
df_unstructured_with_baseline.head(10)

,CR,CRPercent,energyJoule,duration,energyJouleStd,durationStd
0,1.0,0.000000,0.148599,1.291583,0.000759,0.000798
1,2.0,0.500000,0.146279,1.289463,0.000666,0.000318
2,3.0,0.666667,0.144072,1.289303,0.001220,0.000459
3,4.0,0.750000,0.145729,1.289603,0.001565,0.000753
4,8.0,0.875000,0.146867,1.296727,0.001377,0.002831
5,16.0,0.937500,0.147340,1.300994,0.001482,0.003280
6,24.0,0.958333,0.147487,1.303154,0.000945,0.002559
7,32.0,0.968750,0.149061,1.303919,0.001925,0.002522
8,48.0,0.979167,0.151944,1.307653,0.000719,0.001325
9,64.0,0.984375,0.153841,1.308746,0.001488,0.001437


In [105]:
df_structured_with_baseline.head(10)

,CR,CRPercent,energyJoule,duration,energyJouleStd,durationStd
0,1.0,0.000000,0.148599,1.291583,0.000759,0.000798
1,2.0,0.500000,0.080655,0.713011,0.000902,0.000259
2,3.0,0.666667,0.072084,0.608583,0.000836,0.000248
3,4.0,0.750000,0.060929,0.517593,0.000524,0.000231
4,8.0,0.875000,0.042881,0.360626,0.000154,0.000133
5,16.0,0.937500,0.028350,0.246844,0.000184,0.000125
6,24.0,0.958333,0.015708,0.131736,0.000145,0.000072
7,32.0,0.968750,0.015216,0.131455,0.000147,0.000076
8,48.0,0.979167,0.008559,0.073270,0.000061,0.000076
9,64.0,0.984375,0.003549,0.027983,0.000013,0.000020


In [108]:
df_unstructured_speedup.head(10)

,CR,CRPercent,energyJoule,duration,energyJouleStd,durationStd
0,1.0,0.000000,0.141221,1.313500,0.001964,0.000828
1,2.0,0.500000,0.106643,0.935301,0.001017,0.000256
2,3.0,0.666667,0.093900,0.808404,0.000932,0.000108
3,4.0,0.750000,0.087660,0.745093,0.001142,0.000114
4,8.0,0.875000,0.076472,0.651904,0.000856,0.000243
5,16.0,0.937500,0.073072,0.604289,0.000711,0.000139
6,24.0,0.958333,0.070359,0.588659,0.000781,0.000113
7,32.0,0.968750,0.069161,0.579959,0.000800,0.000069
8,48.0,0.979167,0.069393,0.572150,0.000528,0.000067
9,64.0,0.984375,0.068426,0.568698,0.000526,0.000081


## Export data

In [109]:
df_unstructured_with_baseline.to_csv(path_or_buf="../../../src/mnist/train/models/unstructured_pruning/inference.csv", sep=';', index=None)
df_structured_with_baseline.to_csv(path_or_buf="../../../src/mnist/train/models/structured_pruning/inference.csv", sep=';', index=None)
df_unstructured_speedup.to_csv(path_or_buf="../../../src/mnist/train/models/unstructured_pruning/inference_speedup.csv", sep=';', index=None)